## 다층 퍼셉트론을 구현하고, 딥 러닝을 통해서 숫자 필기 데이터를 분류해본다.

### 1. 숫자 필기 데이터 소개

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
digits = load_digits() # 1,979개의 이미지 데이터 로드

ModuleNotFoundError: No module named 'matplotlib'